In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
from skimage.transform import resize
from random import shuffle


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

ImportError: No module named 'PIL'

In [ ]:
list_paths = []
for subdir, dirs, files in os.walk("../input"):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file
        list_paths.append(filepath)
        

In [ ]:
list_train = [filepath for filepath in list_paths if "train/" in filepath]
shuffle(list_train)
list_test = [filepath for filepath in list_paths if "test/" in filepath]

list_train = list_train
list_test = list_test
index = [os.path.basename(filepath) for filepath in list_test]



In [ ]:
list_classes = list(set([os.path.dirname(filepath).split(os.sep)[-1] for filepath in list_paths if "train" in filepath]))


In [ ]:
list_classes = ['Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']


In [ ]:
def get_class_from_path(filepath):
    return os.path.dirname(filepath).split(os.sep)[-1]

def read_and_resize(filepath):
    im_array = np.array(Image.open((filepath)), dtype="uint8")
    pil_im = Image.fromarray(im_array)
    new_array = np.array(pil_im.resize((256,256)))
    return new_array/255

def label_transform(labels):
    labels = pd.get_dummies(pd.Series(labels))
    label_index = labels.columns.values

    return labels, label_index

    

In [ ]:
X_train = np.array([read_and_resize(filepath) for filepath in list_train])
X_test = np.array([read_and_resize(filepath) for filepath in list_test])


In [ ]:
labels = [get_class_from_path(filepath) for filepath in list_train]
y, label_index = label_transform(labels)
y = np.array(y)


In [ ]:
from keras.callbacks import LearningRateScheduler, EarlyStopping 
from keras.callbacks import ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
#from keras.layers import Input 
#from keras.layers import BatchNormalization, GlobalMaxPool2D, Concatenate
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
#input_shape = (256, 256, 3)
nclass = len(label_index)


def get_model():
    
    model = Sequential()
    #conv 1
    model.add(Convolution2D(32, kernel_size=4, border_mode= 'valid', input_shape=(256,256,3), activation= 'relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #conv 2
    model.add(Convolution2D(48,kernel_size=5, activation= 'relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #conv 3
    model.add(Convolution2D(64,kernel_size=5, activation= 'relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #conv 4
    model.add(Convolution2D(128,kernel_size=5, activation= 'relu' ))
    
    #model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    #model.add(Convolution2D(64, 3, 3, activation= 'relu' ))
    #model.add(Convolution2D(32, 3, 3, activation= 'relu' ))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.3))
    model.add(Flatten()) 
    #model.add(Dense(228, activation= 'relu' ))
    model.add(Dense(128, activation= 'relu' ))  
    model.add(Dense(nclass, activation= 'softmax' )) 
    model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    return model
    
    
model = get_model()
model.summary()

file_path="weights.best.hdf5"
# check will store the best model weight which will monitor the validation accuracy not train accuracy
check  = ModelCheckpoint(file_path, monitor = 'val_acc', verbose=1, save_best_only=True, mode='max' )
#early = EarlyStopping(monitor="val_acc", mode="max", patience=1)

checkpoints = [check]

#model.fit(x, y_train, validation_split = 0.3,epochs=25, batch_size=30,verbose=2, callbacks = checkpoints)
history = model.fit(X_train, y, validation_split=0.3, epochs=10, shuffle=True,batch_size=30, verbose=2,
                              callbacks=checkpoints)

'''
model = get_model()
file_path="weights.best.hdf5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor="val_acc", mode="max", patience=1)

callbacks_list = [checkpoint, early] #early

history = model.fit(X_train, y, validation_split=0.1, epochs=3, shuffle=True, verbose=2,
                              callbacks=callbacks_list)
'''

#print(history)



In [ ]:
model.load_weights(file_path)

predicts = model.predict(X_test)
predicts = np.argmax(predicts, axis=1)
predicts = [label_index[p] for p in predicts]

df = pd.DataFrame(columns=['fname', 'camera'])
df['fname'] = index
df['camera'] = predicts
df.to_csv("sub.csv", index=False) 